# Fractopo – Fracture Network Analysis

Notebook that streamlines analysis when branches and nodes are not determined by user choice.

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path
from shutil import rmtree, make_archive
from fractopo.analysis.network import Network
import matplotlib.pyplot as plt
import geopandas as gpd

plt.close()

## Data

Trace and target area data required. The paths can be urls to GeoJSON or local file paths to spatial filetypes (e.g. shapefile, geopackage). The name is used in plot labels and titles. 

1. Pass paths to your **validated** trace and area data here and name the analysis. E.g.,

``` {python}
trace_data = "traces.gpkg"
area_data = "target_area.gpkg"
name = "my-analysis-name"
```
   * The path is relative to the notebook directory. To make things easy you should've copied the notebook the working directory which       either directly contains your trace and area data or has the folder that does. Tab-completion works here aswell.
    
   * Note that the analysis name is used to create a folder like such `results/my-analysis-name` where all analysis results are saved to. If such a folder exists, all contents        will be overridden in the `my-analysis-name` folder.
   * Note that the path is inside quotes. These are mandatory.

In [ ]:
trace_data = ""
area_data = ""
name = ""

The defaults in the next cell are only applied if no parameters are given to the above cell. This will result in a **default** analysis of a trace and area data downloaded from the urls.

In [ ]:
if len(trace_data) == 0:
    # Set defaults
    # Trace and target area data available on GitHub
    trace_data = "data/KB11_traces.geojson"
    area_data = "data/KB11_area.geojson"
    # Name the dataset
    name = "KB11"

2. The preselected analysis set can now be run! To run the notebook, click on the double-right-arrow on at the top of the notebook below the tab bar and click Restart.

   * You can see the cells being executed with numbers appearing on the left.
   * Some cells will take much longer than others depending on code execution time.
   * Scroll down the notebook as the numbers appear until all cells have been reached.
   * If the analysis throws errors they will appear in big red boxes.
  
**However**, you might want to change some defaults such as azimuth set ranges and set names and contour grid cell width. Scroll down to headers with ``USER INPUT:`` prefixes and follow the instructions there to configure default values.

3. If no errors occur during running the results of the analysis will be in `results/my-analysis-name` folder (and an archived .zip).

   * The folder will contain plots and spatial data files:
   
       * Rose plot of trace azimuths, length-weighted
       * Length distribution plots
       * XYI-plots
       * Branches and nodes
       * Contour grids
       * Etc.
   
   * The folder has been also archived as a .zip file for easy downloading (`results/my-analysis-name.zip`).
   
   * If errors do occur:
       
       * Check the error message that occurred for possible solutions.
       * Check that the trace and area paths are correct.
       * You can restart the run from the same double-right-arrow symbol.
       * Report errors that you can't solve at https://github.com/nialov/fractopo-help/issues

4. Some analyses will be run with default settings which might not fit your dataset.

    * This is especially the case for contour grids (grid cell size).
    * Scroll down to the contour grid section to configure if the results are not to your liking.

In [ ]:
# Make/overwrite results dir
results_dir = Path("results") / f"{name}_no_topology"
if results_dir.exists():
    rmtree(results_dir)
results_dir.mkdir(parents=True)

In [ ]:
# Use geopandas to load data from urls/paths
traces = gpd.read_file(trace_data)
area = gpd.read_file(area_data)

In [ ]:
area.total_bounds

In [ ]:
def focus_plot_to_bounds(ax, total_bounds):
    """Focus plot to given bounds."""
    xmin, ymin, xmax, ymax = total_bounds
    extend_x = (xmax - xmin) * 0.05
    extend_y = (ymax - ymin) * 0.05
    ax.set_xlim(xmin - extend_x, xmax + extend_x)
    ax.set_ylim(ymin - extend_y, ymax + extend_y)
    return ax


def save_fig(fig, results_dir: Path, name: str):
    """Save figure as svg image to results dir."""
    fig.savefig(results_dir / f"{name}.svg", bbox_inches="tight")


def remove_duplicate_caseinsensitive_columns(columns) -> set:
    """Remove duplicate columns case-insensitively."""
    lower_case_columns = set(column.lower() for column in columns)
    new_cols = set(columns)
    for column in columns:
        if column not in lower_case_columns and column.lower() in columns:
            print(f"Removing column ({column}) ")
            new_cols.remove(column)
    return new_cols


def as_gpkg_and_shp(geodataframe, name, results_dir: Path = results_dir):
    """Save geodataframe as GeoPackage and as shapefile."""
    non_dupl_columns = remove_duplicate_caseinsensitive_columns(geodataframe.columns)
    fid_col = "fid"
    for column in geodataframe.columns:
        if column not in non_dupl_columns:
            print(f"Dropping column: {column}")
            geodataframe.drop(columns=[column], inplace=True)
        if column.lower() == fid_col:
            # Remove fid columns
            print(f"Dropping column: {column} due to case-insensitive match to fid.")
            geodataframe.drop(columns=[column], inplace=True)
    geodataframe.to_file(results_dir / f"{name}.gpkg", driver="GPKG")
    shp_dir = results_dir / f"{name}_as_shp"
    shp_dir.mkdir()
    geodataframe.to_file(shp_dir / f"{name}.shp")

## Visualizing trace map data

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))
traces.plot(ax=ax, color="blue")
area.boundary.plot(ax=ax, color="red")
ax = focus_plot_to_bounds(ax, area.total_bounds)
save_fig(fig, results_dir, "base_visualization")

## Create Network

### USER INPUT: Pass your own azimuth sets

You may pass your own azimuth sets here for e.g., cross-cutting and abutting relationship analysis. 

You must pass two types of values:

1. Pass a range e.g., `(0, 60)` means the set contains lines with azimuths between 0 and 60 degrees.

    * The range can circle around zero e.g. a range `(170, 30)` is accepted

2. Pass the name for the range. Short names, possibly numerical are preferred e.g., `"1"` or `"A"`.
3. Follow the below shown format and do not remove parenthesis or quotes. The inputs must be valid Python code.

Each range must have an associated name. The inputted ranges must be in the same order as the names.

#### Examples:

Contains three sets:

``` {python}
azimuth_set_ranges = (
    (0, 60),
    (60, 120),
    (120, 180),
)
azimuth_set_names = (
    "1", 
    "2", 
    "3",
)
```

Contains two sets:

``` {python}
azimuth_set_ranges = (
    (0, 60),
    (170, 30),
)
azimuth_set_names = (
    "A", 
    "B",
)
```

In [ ]:
# These are the default values. Input your values here and change the defaults (if needed).
azimuth_set_ranges = (
    (0, 60),
    (60, 120),
    (120, 180),
)
azimuth_set_names = (
    "1",
    "2",
    "3",
)

This next cell automatically checks your azimuth set inputs for basic errors.

In [ ]:
assert len(azimuth_set_ranges) == len(azimuth_set_names)
for set_range in azimuth_set_ranges:
    assert len(set_range) == 2
    assert isinstance(set_range, tuple)

assert all([isinstance(val, str) for val in azimuth_set_names])

In [ ]:
# Create Network and automatically determine branches and nodes
network = Network(
    traces,
    area,
    name=name,
    # NOTE: branches and nodes not determined
    determine_branches_nodes=False,
    snap_threshold=0.001,
    azimuth_set_ranges=azimuth_set_ranges,
    azimuth_set_names=azimuth_set_names,
    # If the target area is a circle, can be changed to True
    circular_target_area=False,
    # If you do not want to crop traces to the target area, pass False here:
    truncate_traces=True,
)

## Rose plots

In [ ]:
# Plot azimuth rose plot of fracture traces
azimuth_bin_dict, fig, ax = network.plot_trace_azimuth()
save_fig(fig, results_dir, "trace_length_weighted_rose_plot")

In [ ]:
network.trace_azimuth_set_counts

## Length distributions

### Trace length distribution

In [ ]:
# Fit for traces
fit_traces = network.trace_lengths_powerlaw_fit()

In [ ]:
# Plot length distribution fits (powerlaw, exponential and lognormal) of fracture traces
fit, fig, ax = network.plot_trace_lengths()
save_fig(fig, results_dir, "trace_length_distribution_fits")

In [ ]:
def describe_powerlaw_fit(fit, network, line_type):
    # Fit properties
    print(f"Automatically determined powerlaw cut-off: {fit.xmin}")
    print(f"Powerlaw exponent: {fit.alpha - 1}")
    description = getattr(network, f"{line_type}_lengths_powerlaw_fit_description")
    # TODO: Bug in fractopo, branch description is not labeled as a property (2.2.2022)
    description = description if not callable(description) else description()

    proportion = description[f"{line_type} lengths cut off proportion"]
    print(f"Proportion of data cut off by cut off: {proportion}")
    comparison = (
        description[f"{line_type} power_law vs. lognormal R"],
        description[f"{line_type} power_law vs. lognormal p"],
    )
    print(f"Compare powerlaw fit to lognormal: R, p = {comparison}")

In [ ]:
describe_powerlaw_fit(fit_traces, network, "trace")

## Data to files

In [ ]:
# Save traces, branches and nodes.
as_gpkg_and_shp(network.trace_gdf, "traces")

In [ ]:
# Zip the folder in results.
base_zip_path = Path("results") / f"{name}"
full_zip_path = base_zip_path.with_suffix(".zip")
if full_zip_path.exists():
    full_zip_path.unlink()
make_archive(base_zip_path, "zip", results_dir)